In [ ]:
import scanpy as sc
import squidpy as sq
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import Counter
from matplotlib_scalebar.scalebar import ScaleBar
import os

import cell2location
import scvi

DPI = 300
FONTSIZE = 20  # 42
sc.settings.set_figure_params(
    scanpy=True, dpi=60, transparent=True, vector_friendly=True, dpi_save=DPI
)
from matplotlib import rcParams

rcParams["pdf.fonttype"] = 42

import seaborn as sns
from pathlib import Path

custom_params = {"font.size": FONTSIZE, "figure.dpi": 100, "savefig.dpi": DPI}
sns.set_theme(style="ticks", rc=custom_params)

# QC utils functions - a package
from vistools import utils
from vistools import spatial_plots

In [ ]:
SAMPLE_NAME = "CRC08_LM_VISIUM"

# DIR2SAVE
DIR2SAVE = Path(
    f"/data/BCI-CRC/nasrine/data/CRC/spatial/CRC_LM_VISIUM/CRC_LM_VISIUM_04_08_09_11/cell2location/{SAMPLE_NAME}"
)
DIR2SAVE.mkdir(parents=True, exist_ok=True)

In [ ]:
run_name = DIR2SAVE.joinpath(f"cell2location_map-no_cycling_TME/")
run_name.mkdir(parents=True, exist_ok=True)

# figures
FIG2SAVE = run_name.joinpath("figures/final/")
FIG2SAVE.mkdir(parents=True, exist_ok=True)
# set the global variable: sc.settings.figdir to save all plots
sc.settings.figdir = FIG2SAVE

In [ ]:
adata_file = f"{run_name}/sp.h5ad"
adata_vis = sc.read_h5ad(adata_file)
mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)

In [ ]:
# add 5% quantile, representing confident cell abundance, 'at least this amount is present',
# to adata.obs with nice names for plotting
adata_vis.obs[adata_vis.uns["mod"]["factor_names"]] = adata_vis.obsm[
    "q05_cell_abundance_w_sf"
]
adata_vis.uns["mod"]["factor_names"]

In [ ]:
gene_dict = {
    "Stem": ["LGR5", "ASCL2", "SMOC2"],
    "Stem (NOTUM high)": ["NOTUM", "APCDD1", "NKD1"],
    "pEMT": ["KRT17", "EMP1", "MMP7", "PLAUR", "KRT80", "LAMC2", "L1CAM", "IFI6", "ANXA1", "LAMA3", "ECM1", 
             "IFI16"],
    "Hypoxia": ["VEGFA", "ENO2"],
    "Colonocyte": ["FABP1", "CA2", "SLC26A3"],
    "Cancer": ["GRHL2", "CDH1", "ELF3"],
    "pEMT_subset": ["KRT17", "EMP1",  "LAMC2", "PLAUR", "LAMA3", "ANXA1", "TACSTD2", "IFIT3", ],
    "Fetal": ["ANXA1", "TACSTD2"],
    "IFIT": ["IFI6", "IFI44L", "IFI44", "IFI16", "IFIH1", "IFIT3", "IFIT1"],
    "Myeloid": ["CD68", "CD14"],
    "Stromal": ["DCN", "COL1A1", "COL3A1"],
    "Endothelial": ["PECAM1", "CDH5"],
    "T": ["TRAC", "CD3D", "CD3E", "KLRB1"],
    "B": ["CD79A", "MS4A1", "CD19"],
    "Plasma": ["MZB1", "JCHAIN"],
    "Hepatocyte": ["ALB", "TTR", "APOA1"],
    "Differentiation": ["KRT20"],
    "Epithelial": ["EPCAM"],
    "Desmoplastic rim": ["FAP"], # from petervermeulenn paper 
    "Immunosuppressive fibro": ["LOXL2", "STAT3"],
    "Goblet": ["MUC2"],
    #"Enteroendocrine": ["CHGA","CHGB"]

    #"Enteroendocrine": ["CHGA","CHGB"]
}

In [ ]:
# keep raw
adata_vis.layers["raw"] = adata_vis.X.copy()  # preserve counts

# normalize + log1p
sc.pp.normalize_total(adata_vis, target_sum=1e4, inplace=True)
adata_vis.layers["normalised"] = adata_vis.X.copy()
sc.pp.log1p(adata_vis)

adata_vis.layers["log1p"] = adata_vis.X.copy()

adata_vis.raw = adata_vis  # keep normalised log1p

In [ ]:
for cellstate in gene_dict.keys():
    
    for visium_slide in set(adata_vis.obs["Sample"]):
        print(visium_slide)

        # select one slide
        slide = utils.select_slide(adata_vis, s=visium_slide, batch_key="Sample")
        
        sc.pl.spatial(slide, use_raw=True, cmap='RdPu',color=gene_dict[cellstate],wspace=0.15,hspace=0.4,
                      size=1.3, img_key='hires',alpha_img=0.3, vmin=0, vmax='p99.7',
                      save=f"{visium_slide}_{cellstate}_marker_genes_geneexp.pdf"
                     )

In [ ]:
for cellstate in gene_dict.keys():
    
    for visium_slide in set(adata_vis.obs["Sample"]):
        print(visium_slide)

        # select one slide
        slide = utils.select_slide(adata_vis, s=visium_slide, batch_key="Sample")
        
        sc.pl.spatial(slide, layer="raw", cmap='RdPu',color=gene_dict[cellstate],wspace=0.15,hspace=0.4,
                      size=1.3, img_key='hires',alpha_img=0.3, vmin=0, vmax='p99.2',
                      save=f"{visium_slide}_{cellstate}_marker_genes_mRNAcount.pdf"
                     )

### plot for 1 sample cell abundance and gene exp of marker genes and mRNA counts

In [ ]:
set(adata_vis.obs["Sample"])

In [ ]:
adata_vis.obs["Sample"].cat.categories

In [ ]:
# select one slide
samp2plot = adata_vis.obs["Sample"].cat.categories[0] #"ST_LM_4"
slide = utils.select_slide(adata_vis, s=samp2plot, batch_key="Sample")

In [ ]:
# gene exp
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15,4))
sc.pl.spatial(slide,cmap='Blues',color='ipEMT', ncols=3, wspace=0,
              size=1.3,img_key='hires',alpha_img=0.3,vmin=0, vmax='p99.5',legend_loc=None,
             show=False,ax=axs[0],title='ipEMT')
sc.pl.spatial(slide,cmap='RdPu',color=['EMP1'], ncols=3, wspace=0,
              size=1.3,img_key='hires',alpha_img=0.3,vmin=0, vmax='p99.7',legend_loc=None,
             show=False,ax=axs[1],title='EMP1', use_raw=True) # 
sc.pl.spatial(slide,cmap='RdPu',color=['LAMC2'], ncols=3, wspace=0,
              size=1.3,img_key='hires',alpha_img=0.3,vmin=0, vmax='p99.7',legend_loc=None,
             show=False,ax=axs[2],title='LAMC2', use_raw=True)
axs[0].set_ylabel('')
axs[0].set_xlabel('')
axs[1].set_ylabel('')
axs[1].set_xlabel('')
axs[2].set_ylabel('')
axs[2].set_xlabel('')
plt.tight_layout(pad=0.000)

#fig.savefig(FIG2SAVE.joinpath('{FIG2SAVE}.pdf'))
path2save = os.path.join(FIG2SAVE, f"{samp2plot}_ipEMT_genexp_2genes.pdf")
print("Saving figure into ", path2save)
fig.savefig(path2save, dpi=DPI, format='pdf', facecolor='w', transparent=True, edgecolor='w',
            orientation='landscape', bbox_inches="tight")
    
fig.show() 

In [ ]:
# mRNA count
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(22,4))
sc.pl.spatial(slide,cmap='Blues',color='ipEMT', ncols=3, wspace=0,
              size=1.3,img_key='hires',alpha_img=0.3,vmin=0, vmax='p99.5',legend_loc=None,
             show=False,ax=axs[0],title='ipEMT')
sc.pl.spatial(slide,cmap='RdPu',color=['EMP1'], ncols=3, wspace=0,
              size=1.3,img_key='hires',alpha_img=0.3,vmin=0, vmax='p99.5',legend_loc=None,
             show=False,ax=axs[1],title='EMP1', layer="raw") # 
sc.pl.spatial(slide,cmap='RdPu',color=['LAMC2'], ncols=3, wspace=0,
              size=1.3,img_key='hires',alpha_img=0.3,vmin=0, vmax='p99.5',legend_loc=None,
             show=False,ax=axs[2],title='LAMC2', layer="raw")
axs[0].set_ylabel('')
axs[0].set_xlabel('')
axs[1].set_ylabel('')
axs[1].set_xlabel('')
axs[2].set_ylabel('')
axs[2].set_xlabel('')
plt.tight_layout(pad=0.000)

#fig.savefig(FIG2SAVE.joinpath('{FIG2SAVE}.pdf'))
path2save = os.path.join(FIG2SAVE, f"{samp2plot}_ipEMT_mRNAcount_2genes.pdf")
print("Saving figure into ", path2save)
fig.savefig(path2save, dpi=DPI, format='pdf', facecolor='w', transparent=True, edgecolor='w',
            orientation='landscape', bbox_inches="tight")
    
fig.show() 

In [ ]:
sc.pl.spatial(slide, layer="raw", cmap='RdPu',
              color=["LGR5", "NOTUM", "EMP1", "LAMC2", "VEGFA"],
              wspace=0.15,hspace=0.4,ncols=5,
                      size=1.3, img_key='hires',alpha_img=0.3, vmin=0, vmax='p99.2',
                      save=f"{samp2plot}_genes2show_mRNAcount.pdf"
                     )

sc.pl.spatial(slide, use_raw="True", cmap='RdPu',
              color=["LGR5", "NOTUM", "EMP1", "LAMC2", "VEGFA"],
              wspace=0.15,hspace=0.4,ncols=5,
                      size=1.3, img_key='hires',alpha_img=0.3, vmin=0, vmax='p99.2',
                      save=f"{samp2plot}_genes2show_geneexp.pdf"
                     )